In [23]:
import numpy as np

class Traffic:
    """
    A Nagel-Schreckenberg traffic cellular automaton simulation.

    The road is represented by a 1D array , where each cell is either empty (-1)
    or occupied by a car with a certain velocity (integer > 0).

    Parameters:
    n (int): Length of the road (number of cells)
    density (float): Fraction of cells initially containing cars (0 to 1)
    vmax (int): Maximum allowed velocity
    p (float): Random slowdown probability (0 to 1)
    grid (np.ndarray): The grid of car velocities or -1 for empty
    history (list): A list of configurations of the grid at each step
    """

    def __init__(self, n=100, density=0.3, vmax=5, p=0.2, random_state=None):
        np.random.seed(random_state)

        self.n = n
        self.vmax = vmax
        self.p = p
        
        # Random initial placement of cars
        grid = -1 * np.ones(n, dtype=int)
        car_positions = np.random.choice(np.arange(n), size=int(density*n), replace=False)
        for pos in car_positions:
            grid[pos] = np.random.randint(1, vmax+1)  

        self.grid = grid
        self.history = [self.grid.copy()] 

    def step(self):
        """
        Performs a single step of the Nagel-Schreckenberg model.
        Each step consists of four actions applied in parallel:
        1. Acceleration
        2. Slowing down due to cars ahead
        3. Randomization (slow down with probability p)
        4. Car motion (cars move forward by their velocities)
        """
        grid = self.grid.copy()
        new_grid = -1 * np.ones_like(grid)

        # 1. Acceleration
        velocities = np.where(grid >= 0, np.minimum(grid + 1, self.vmax), -1)

        # 2. Slowing down
        for i in range(self.n):
            if velocities[i] >= 0: 
                distance = 1
                while grid[(i + distance) % self.n] == -1 and distance <= self.vmax:
                    distance += 1
                distance -= 1  
                velocities[i] = min(velocities[i], distance)

        # 3. Randomization
        rand = np.random.rand(self.n)
        velocities = np.where((velocities > 0) & (rand < self.p), velocities - 1, velocities)

        # 4. Car motion 
        for i in range(self.n):
            v = velocities[i]
            if v >= 0:
                new_pos = (i + v) % self.n
                new_grid[new_pos] = v

        self.grid = new_grid
        self.history.append(self.grid.copy())
        return self.grid
    
    def average_flow(self):
        """
        Compute the average flow: 
        flow = density * average velocity = (#cars / n) * mean velocity of cars
        """
        cars = self.grid[self.grid >= 0]          
        if len(cars) == 0:
            return 0
        return (len(cars) / self.n) * np.mean(cars)


    def jam_fraction(self):
        """
        Fraction of cars that are part of a jam.
        A 'jam' is defined as a car with velocity 0.
        """
        cars = self.grid[self.grid >= 0]
        if len(cars) == 0:
            return 0
        return np.sum(cars == 0) / len(cars)


    def simulate(self, n_step):
        """Run the automaton for n_step iterations."""
        for _ in range(n_step):
            self.step()
        return self.grid


In [24]:
model = Traffic(
    n=100,        
    density=0.3,  # I have been keeping it around 0.3 even around 0.5 you get almost a complete traffic jam
    vmax=5,       
    p=0.3,        
    random_state=42
)

model.simulate(200)


array([-1,  1,  0,  0, -1,  1, -1, -1, -1,  2, -1,  0, -1,  0,  0,  0, -1,
        1,  0, -1,  1,  0, -1, -1, -1,  2, -1, -1, -1,  3, -1,  1, -1, -1,
        1, -1, -1, -1,  2, -1, -1, -1, -1, -1, -1, -1,  4, -1, -1, -1, -1,
       -1, -1,  5, -1, -1, -1,  1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  5, -1, -1, -1, -1, -1,  5, -1, -1, -1, -1,
       -1,  5, -1, -1, -1,  0, -1,  1, -1, -1,  2, -1, -1,  0,  0])

In [25]:
# Test 1: empty road -> flow = jam = 0
model = Traffic(n=20, density=0, vmax=5, p=0)
model.simulate(10)

flow = model.average_flow()
jam = model.jam_fraction()

print("Test 1: Empty Road")
print(f"  Expected flow = 0, Actual flow = {flow}")
print(f"  Expected jam fraction = 0, Actual jam fraction = {jam}")
print("  PASS" if flow == 0 and jam == 0 else "  FAIL")
print()


# Test 2: full road → most cars must jam jam = 1
model = Traffic(n=20, density=1, vmax=5, p=0)
model.simulate(10)

jam = model.jam_fraction()

print("Test 2: Full Road")
print(f"  Expected jam fraction = 1, Actual jam fraction = {jam:.3f}")
print("  PASS" if jam > 0.9 else "  FAIL")





Test 1: Empty Road
  Expected flow = 0, Actual flow = 0
  Expected jam fraction = 0, Actual jam fraction = 0
  PASS

Test 2: Full Road
  Expected jam fraction = 1, Actual jam fraction = 1.000
  PASS
